In [1]:
#!pip install pymc4

In [2]:
#!pip install aesara

In [3]:
#!pip install prophet

Carryover effect
Mathematical functions can be used to model the carryover effect
- geodecay transformation
- Adstock transformation


In [4]:

#!pip install plotnine

In [5]:
import pandas as pd
import numpy as np
import math
import sys
import os


In [6]:
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 18
import seaborn as sns

from scipy.stats.mstats import mquantiles

from plotnine import * 

In [7]:
import pymc3 as pm
import arviz as az

#import aesara
#import aesara.tensor as at
import theano.tensor as tt

In [8]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

In [9]:
from scipy import optimize
from prophet import Prophet

In [10]:
data = pd.read_csv("/Users/mahrukhwaqar/datascience/recruitment_challenge/MMM_test_data.csv")

In [11]:
data

,start_of_week,revenue,spend_channel_1,spend_channel_2,spend_channel_3,spend_channel_4,spend_channel_5,spend_channel_6,spend_channel_7
0,30-08-20,157906.75,2625.48,262.71,12954.12,3609.63,12955.29,12659.12,19379.79
1,06-09-20,186425.68,2634.01,108.66,8760.28,4560.60,12747.70,12338.18,22473.45
2,13-09-20,161607.39,2087.08,110.32,7155.42,4362.96,15015.41,10811.15,22596.05
3,20-09-20,180089.13,1690.70,52.79,15185.22,3883.41,15521.41,12890.22,24728.73
4,27-09-20,217793.98,1547.30,80.56,18524.05,4043.09,15793.74,12642.55,26515.48
...,...,...,...,...,...,...,...,...,...
99,24-07-22,72021.50,0.00,1118.78,22305.10,4566.01,2987.70,0.00,19916.88
100,31-07-22,90786.21,0.00,122.16,15812.65,4795.51,3293.40,0.00,22687.98
101,07-08-22,105929.40,0.00,754.09,12166.85,4961.12,4057.03,0.00,30142.64
102,14-08-22,90506.31,0.00,1043.22,10856.58,4431.13,3086.81,0.00,28622.78


In [12]:
#print(type('revenue'))
print(data.dtypes)

start_of_week       object
revenue            float64
spend_channel_1    float64
spend_channel_2    float64
spend_channel_3    float64
spend_channel_4    float64
spend_channel_5    float64
spend_channel_6    float64
spend_channel_7    float64
dtype: object


In [13]:
#data['revenue'] = data['revenue'].astype(float)

In [14]:
print(type('spend_channel_2'))

<class 'str'>


In [15]:
X = data.drop(columns=['revenue', 'start_of_week'])
y = ["revenue"]

In [16]:
def saturate(x, a):
    return 1 - tt.exp(-a*x)

def carryover(x, strength, length=21):
    w = tt.as_tensor_variable(
        [tt.power(strength, i) for i in range(length)]
    )
    
    x_lags = tt.stack(
        [tt.concatenate([
            tt.zeros(i),
            x[:x.shape[0]-i]
        ]) for i in range(length)]
    )
    
    return tt.dot(w, x_lags)

In [17]:

with pm.Model() as mmm:
    channel_contributions = []
    
    for channel in X.columns:
        coef = pm.Exponential(f'coef_{channel}', lam=0.0001)
        sat = pm.Exponential(f'sat_{channel}', lam=1)
        car = pm.Beta(f'car_{channel}', alpha=2, beta=2)
        
        channel_data = X[channel].values
        channel_contribution = pm.Deterministic(
            f'contribution_{channel}',
            coef * saturate(
                carryover(
                    channel_data,
                    car
                ),
                sat
            )
        )
        
        channel_contributions.append(channel_contribution)
        
    base = pm.Exponential('base', lam=0.0001)
    noise = pm.Exponential('noise', lam=0.0001)
    
    sales = pm.Normal(
        'revenue',
        mu=sum(channel_contributions) + base,
        sigma=noise,
        observed=y
    )
    
    trace = pm.sample(return_inferencedata=True, tune=3000)

ValueError: could not convert string to float: 'revenue'

In [ ]:
az.plot_posterior(
    trace,
    var_names=['~contribution'],
    filter_vars='like'
)